# Spline fitting for μ_max estimation

Evaluation on synthetic data modelled from real growth curves.

In [239]:
import numpy as np
from scipy.interpolate import UnivariateSpline
import warnings
from pathlib import Path
import re
import zipfile
import xml.etree.ElementTree as ET

import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

PLOTLY_TEMPLATE = "plotly_white"
PLOTLY_CONFIG = {"displayModeBar": True}

COLORS = {
    "data": "#888888",
    "true": "#000000",
    "fixed": "red",
    "auto": "blue",
    "rough": "blue",
}


def _dash(style):
    return {"-": "solid", "--": "dash", "-.": "dashdot", ":": "dot"}.get(style, "solid")


def _row_col(i, n_cols):
    return (i // n_cols) + 1, (i % n_cols) + 1

In [240]:
import sys

# Import growth models from src/growthcurves/models.py (no model redefinition in notebook).
if "growthcurves.models" not in sys.modules:
    if "growthcurves" not in sys.modules:
        for _base in [Path.cwd(), Path.cwd().parent]:
            _src = _base / "src"
            if (_src / "growthcurves" / "models.py").exists():
                if str(_src) not in sys.path:
                    sys.path.insert(0, str(_src))
                break

from growthcurves.models import (
    phenom_gompertz_model,
    phenom_logistic_model,
    phenom_richards_model,
)


def make_growth_curve(
    t,
    N0=0.05,
    Nmax=2.0,
    mu_max=0.5,
    lag=4.0,
    noise_sigma=0.05,
    seed=42,
    model="gompertz",
    richards_nu=1.2,
):
    """
    Noisy synthetic growth curve from imported growth models.

    Multiplicative lognormal noise (additive in log-space).
    Supported models: gompertz, logistic, richards.
    """
    t = np.asarray(t, dtype=float)
    rng = np.random.default_rng(seed)

    N0 = float(max(N0, 1e-10))
    Nmax = float(max(Nmax, N0 * 1.01))
    A = float(np.log(max(Nmax, 1e-12) / max(N0, 1e-12)))
    if (not np.isfinite(A)) or (A <= 0):
        A = 1e-6

    model_key = str(model).strip().lower()
    if model_key == "gompertz":
        N_true = phenom_gompertz_model(t=t, A=A, mu_max=float(mu_max), lam=float(lag), N0=N0)
    elif model_key == "logistic":
        N_true = phenom_logistic_model(t=t, A=A, mu_max=float(mu_max), lam=float(lag), N0=N0)
    elif model_key == "richards":
        nu = float(np.clip(richards_nu, 0.05, 5.0))
        N_true = phenom_richards_model(
            t=t,
            A=A,
            mu_max=float(mu_max),
            lam=float(lag),
            nu=nu,
            N0=N0,
        )
    else:
        raise ValueError(f"Unknown model '{model}'. Use gompertz/logistic/richards.")

    N_noisy = np.asarray(N_true, dtype=float) * np.exp(rng.normal(0, noise_sigma, size=len(t)))
    return np.asarray(N_true, dtype=float), np.clip(N_noisy, 1e-10, None)


# Synthetic data are generated once in the dedicated eval_curves cell below.


In [241]:
def _second_diff_series(t, y):
    t = np.asarray(t, dtype=float)
    y = np.asarray(y, dtype=float)

    if len(t) < 4 or np.ptp(t) <= 0:
        return np.array([], dtype=float)

    order = np.argsort(t, kind="mergesort")
    t = t[order]
    y = y[order]

    dt = np.diff(t)
    if len(dt) < 2 or np.any(dt <= 0):
        return np.array([], dtype=float)

    d1 = np.diff(y) / dt
    dt2 = 0.5 * (dt[:-1] + dt[1:])
    if len(dt2) < 1 or np.any(dt2 <= 0):
        return np.array([], dtype=float)

    d2 = np.diff(d1) / dt2
    if len(d2) < 3:
        return np.array([], dtype=float)

    h = float(np.median(dt))
    return np.asarray(d2 * h**2, dtype=float)


def _trim_by_abs_deviation(vals, trim_q=0.80):
    vals = np.asarray(vals, dtype=float)
    vals = vals[np.isfinite(vals)]
    if len(vals) < 3:
        return vals

    if trim_q is None:
        return vals

    q = float(trim_q)
    if (not np.isfinite(q)) or (q <= 0.0) or (q >= 1.0):
        return vals

    abs_dev = np.abs(vals - np.median(vals))
    if len(abs_dev) < 5:
        return vals

    cutoff = float(np.quantile(abs_dev, q))
    core = vals[abs_dev <= cutoff]
    if len(core) >= 3:
        return core
    return vals


def _sigma_second_diff_mad(t, y, trim_q=0.80):
    d2y = _second_diff_series(t, y)
    if len(d2y) < 3:
        return np.nan

    d2y = _trim_by_abs_deviation(d2y, trim_q=trim_q)
    if len(d2y) < 3:
        return np.nan

    mad = float(np.median(np.abs(d2y - np.median(d2y))))
    return float(1.4826 * mad / np.sqrt(6.0))


def _sigma_second_diff_iqr(t, y):
    d2y = _second_diff_series(t, y)
    if len(d2y) < 3:
        return np.nan

    q25, q75 = np.quantile(d2y, [0.25, 0.75])
    iqr = float(q75 - q25)
    if iqr <= 0:
        return np.nan

    return float((iqr / 1.349) / np.sqrt(6.0))


def _sigma_second_diff_std(t, y):
    d2y = _second_diff_series(t, y)
    if len(d2y) < 3:
        return np.nan

    sd = float(np.std(d2y, ddof=1))
    return float(sd / np.sqrt(6.0))


def _sigma_first_diff_mad(t, y, trim_q=0.80):
    t = np.asarray(t, dtype=float)
    y = np.asarray(y, dtype=float)

    if len(t) < 3 or np.ptp(t) <= 0:
        return np.nan

    order = np.argsort(t, kind="mergesort")
    t = t[order]
    y = y[order]

    dt = np.diff(t)
    if len(dt) < 2 or np.any(dt <= 0):
        return np.nan

    dy = np.diff(y)
    slopes = dy / dt
    slope_med = float(np.median(slopes))
    resid = dy - slope_med * dt
    resid = _trim_by_abs_deviation(resid, trim_q=trim_q)
    if len(resid) < 2:
        return np.nan

    mad = float(np.median(np.abs(resid - np.median(resid))))
    return float(1.4826 * mad / np.sqrt(2.0))


def _estimate_sigma_log_noise(t, y, method="second_diff_mad", trim_mad=True, trim_q=0.80):
    method = str(method).strip().lower()
    trim_q_use = float(trim_q) if trim_mad else None

    if method == "second_diff_mad":
        sigma = _sigma_second_diff_mad(t, y, trim_q=trim_q_use)
    elif method == "first_diff_mad":
        sigma = _sigma_first_diff_mad(t, y, trim_q=trim_q_use)
    elif method == "second_diff_iqr":
        sigma = _sigma_second_diff_iqr(t, y)
    elif method == "second_diff_std":
        sigma = _sigma_second_diff_std(t, y)
    else:
        raise ValueError(
            "Unknown sigma method. Use one of: second_diff_mad, first_diff_mad, second_diff_iqr, second_diff_std."
        )

    if (not np.isfinite(sigma)) or (sigma < 1e-8):
        sigma = 1e-4
    return float(sigma)


def _robust_sigma_from_second_diff(t, y, trim_q=0.80):
    """
    Backward-compatible default noise estimate in log-space from second divided differences.

    A trimmed MAD is used to reduce inflation when many spikes are present.
    """
    return _estimate_sigma_log_noise(t, y, method="second_diff_mad", trim_mad=True, trim_q=trim_q)


# Multiplier applied to the theoretically optimal n*sigma^2 smoothing factor.
_SMOOTH_MULT = 5.0

# Fixed smoothing factor for the baseline method.
FIXED_S = 0.55


def fixed_s_spline(t, N, s=FIXED_S, k=3):
    """Baseline: smoothing spline with a fixed smoothing factor s in ln(N) space."""
    t_in = np.asarray(t, dtype=float)
    y_in = np.log(np.asarray(N, dtype=float))
    order = np.argsort(t_in, kind="mergesort")
    spline = UnivariateSpline(t_in[order], y_in[order], k=k, s=s)
    return spline


def _auto_s_from_sigma(n, y, sigma, smooth_mult=None, clip_smoothing=True):
    n = int(max(int(n), 2))
    sigma = float(max(float(sigma), 1e-8))

    smooth_mult = float(_SMOOTH_MULT if smooth_mult is None else smooth_mult)
    if not np.isfinite(smooth_mult):
        smooth_mult = float(_SMOOTH_MULT)

    if clip_smoothing:
        smooth_mult_eff = float(np.clip(smooth_mult, 0.25, 30.0))
    else:
        smooth_mult_eff = float(max(smooth_mult, 0.0))

    raw_s = float(smooth_mult_eff * float(n) * sigma**2)

    s_min = 0.01
    s_max = max(s_min * 10, 0.8 * float(n) * float(np.var(y)))

    if clip_smoothing:
        s_opt = float(np.clip(raw_s, s_min, s_max))
    else:
        s_opt = float(max(raw_s, 0.0))

    return s_opt, raw_s, smooth_mult_eff, s_min, s_max


def auto_smooth_spline_variant(
    t,
    N,
    k=3,
    smooth_mult=None,
    sigma_method="second_diff_mad",
    trim_mad=True,
    clip_smoothing=True,
    trim_q=0.80,
):
    """
    Fit a smoothing spline to ln(N) with configurable noise estimation and smoothing rules.
    """
    t_in = np.asarray(t, dtype=float)
    y_in = np.log(np.asarray(N, dtype=float))

    order = np.argsort(t_in, kind="mergesort")
    t = t_in[order]
    y = y_in[order]

    sigma = _estimate_sigma_log_noise(
        t,
        y,
        method=sigma_method,
        trim_mad=bool(trim_mad),
        trim_q=trim_q,
    )

    s_opt, _, _, _, _ = _auto_s_from_sigma(
        n=len(t),
        y=y,
        sigma=sigma,
        smooth_mult=smooth_mult,
        clip_smoothing=bool(clip_smoothing),
    )

    spline = UnivariateSpline(t, y, k=k, s=s_opt)
    return spline, float(s_opt), float(sigma)


def auto_smooth_spline(t, N, k=3, smooth_mult=None):
    """
    Fit a smoothing spline to ln(N) with automatic smoothing factor selection.

    Default behavior keeps backward compatibility with the original method:
    second-difference MAD sigma with MAD trimming and clipping enabled.
    """
    return auto_smooth_spline_variant(
        t,
        N,
        k=k,
        smooth_mult=smooth_mult,
        sigma_method="second_diff_mad",
        trim_mad=True,
        clip_smoothing=True,
        trim_q=0.80,
    )


print("fixed_s_spline(), auto_smooth_spline_variant(), and auto_smooth_spline() defined.")



fixed_s_spline() and auto_smooth_spline() defined.


In [242]:
# ---------------------------------------------------------------------------
# Load real growth curves from xlsx files (used to build synthetic distributions)
# ---------------------------------------------------------------------------

DATA_DIR = Path("/Users/sambra/Documents/GitHub/TheGrowthAnalysisApp/example_data")
MIN_POINTS = 10
MIN_OD_INCREASE = 0.05

# Parameters used for auto spline evaluation on real curves
REAL_SMOOTH_MULT = float(globals().get("REAL_SMOOTH_MULT", _SMOOTH_MULT))
REAL_MU_EDGE_FRAC = float(globals().get("REAL_MU_EDGE_FRAC", 0.0))
REAL_MU_Q = float(globals().get("REAL_MU_Q", 1.0))


def _mu_max_from_spline(spline, t, edge_frac=0.0, q=1.0, n_eval=320):
    t = np.asarray(t, dtype=float)
    t0 = float(np.nanmin(t))
    t1 = float(np.nanmax(t))
    span = max(t1 - t0, 1e-8)
    e = float(np.clip(edge_frac, 0.0, 0.35))
    lo = t0 + e * span
    hi = t1 - e * span
    if hi <= lo:
        lo, hi = t0, t1
    te = np.linspace(lo, hi, int(max(80, n_eval)))
    mu = np.asarray(spline.derivative()(te), dtype=float)
    mu = mu[np.isfinite(mu)]
    if len(mu) == 0:
        return np.nan
    qq = float(np.clip(q, 0.80, 1.0))
    if qq >= 0.999999:
        return float(np.max(mu))
    return float(np.quantile(mu, qq))


def _excel_col_to_idx_auto(cell_ref):
    m = re.match(r"([A-Z]+)", str(cell_ref))
    if not m:
        return 0
    col = m.group(1)
    idx = 0
    for ch in col:
        idx = idx * 26 + (ord(ch) - ord("A") + 1)
    return idx - 1


def _parse_xlsx_first_sheet_auto(path):
    ns_main = {"x": "http://schemas.openxmlformats.org/spreadsheetml/2006/main"}
    ns_rel = {"r": "http://schemas.openxmlformats.org/package/2006/relationships"}

    with zipfile.ZipFile(path) as zf:
        shared = []
        if "xl/sharedStrings.xml" in zf.namelist():
            root_ss = ET.fromstring(zf.read("xl/sharedStrings.xml"))
            for si in root_ss.findall("x:si", ns_main):
                txt = "".join(t.text or "" for t in si.findall(".//x:t", ns_main))
                shared.append(txt)

        wb_root = ET.fromstring(zf.read("xl/workbook.xml"))
        first_sheet = wb_root.find("x:sheets/x:sheet", ns_main)
        if first_sheet is None:
            raise ValueError(f"No sheets found in {path.name}")
        rel_id = first_sheet.attrib.get(
            "{http://schemas.openxmlformats.org/officeDocument/2006/relationships}id"
        )

        rels_root = ET.fromstring(zf.read("xl/_rels/workbook.xml.rels"))
        target = None
        for rel in rels_root.findall("r:Relationship", ns_rel):
            if rel.attrib.get("Id") == rel_id:
                target = rel.attrib.get("Target")
                break
        if target is None:
            raise ValueError(f"Could not resolve first sheet rel in {path.name}")

        sheet_xml = "xl/" + target.lstrip("/")
        root_sheet = ET.fromstring(zf.read(sheet_xml))

        rows_dict = {}
        max_col = 0
        for row in root_sheet.findall("x:sheetData/x:row", ns_main):
            rnum = int(row.attrib.get("r", "1")) - 1
            row_map = {}
            for c in row.findall("x:c", ns_main):
                ref = c.attrib.get("r", "A1")
                cidx = _excel_col_to_idx_auto(ref)
                ctype = c.attrib.get("t")
                val = None
                v_node = c.find("x:v", ns_main)
                is_node = c.find("x:is", ns_main)
                if ctype == "s" and v_node is not None:
                    sid = int(v_node.text)
                    if 0 <= sid < len(shared):
                        val = shared[sid]
                elif ctype == "inlineStr" and is_node is not None:
                    t_node = is_node.find("x:t", ns_main)
                    val = t_node.text if t_node is not None else None
                elif v_node is not None:
                    val = v_node.text
                row_map[cidx] = val
                max_col = max(max_col, cidx)
            rows_dict[rnum] = row_map

        max_row = max(rows_dict.keys()) if rows_dict else 0
        rows = []
        for r in range(max_row + 1):
            row = [None] * (max_col + 1)
            for cidx, v in rows_dict.get(r, {}).items():
                row[cidx] = v
            rows.append(row)

    return rows


def _to_float_auto(v):
    if v is None:
        return np.nan
    if isinstance(v, (int, float, np.floating)):
        return float(v)
    s = str(v).strip().replace(",", ".")
    try:
        return float(s)
    except Exception:
        return np.nan


def _extract_curves_auto(rows, file_name, min_points=10):
    if len(rows) < 3:
        return []
    header = rows[0]
    curves = []
    n_cols = len(header)
    for j in range(1, n_cols):
        well = header[j]
        if well is None or str(well).strip() == "":
            continue
        well = str(well).strip()
        t_vals, y_vals = [], []
        for r in range(1, len(rows)):
            t_raw = rows[r][0] if 0 < len(rows[r]) else None
            y_raw = rows[r][j] if j < len(rows[r]) else None
            t_f = _to_float_auto(t_raw)
            y_f = _to_float_auto(y_raw)
            if np.isfinite(t_f) and np.isfinite(y_f):
                t_vals.append(t_f)
                y_vals.append(y_f)
        if len(t_vals) < min_points:
            continue
        t = np.asarray(t_vals, dtype=float)
        y = np.asarray(y_vals, dtype=float)
        m = np.isfinite(t) & np.isfinite(y) & (y > 0)
        t, y = t[m], y[m]
        if len(t) < min_points or np.ptp(t) <= 0:
            continue
        o = np.argsort(t, kind="mergesort")
        t, y = t[o], y[o]
        if np.nanmax(t) > 96:
            t = t / 60.0
        uniq_t, uniq_idx = np.unique(t, return_index=True)
        t, y = uniq_t, y[uniq_idx]
        if len(t) < min_points or np.ptp(t) <= 0:
            continue
        curves.append({"curve_id": f"{file_name}:{well}", "file": file_name,
                        "well": well, "t": t, "N": y})
    return curves


if not DATA_DIR.exists():
    raise RuntimeError(f"Real-data directory not found: {DATA_DIR}")

data_files = sorted(DATA_DIR.glob("*_data.xlsx"))
if not data_files:
    raise RuntimeError(f"No *_data.xlsx files found in {DATA_DIR}")

real_curves_auto = []
for fp in data_files:
    try:
        rows = _parse_xlsx_first_sheet_auto(fp)
        real_curves_auto.extend(_extract_curves_auto(rows, fp.name, min_points=MIN_POINTS))
    except Exception as exc:
        print(f"Skipping {fp.name}: {exc}")

if not real_curves_auto:
    raise RuntimeError("No usable real curves extracted.")

print(f"Loaded {len(real_curves_auto)} real curves from {len(data_files)} file(s).")

Loaded 226 real curves from 3 file(s).


In [243]:
# ---------------------------------------------------------------------------
# Real-data-informed synthetic data generation
# Build ONE synthetic dataset (eval_curves) and reuse it for all downstream plots.
# ---------------------------------------------------------------------------

if "real_curves_auto" not in globals() or len(real_curves_auto) == 0:
    raise RuntimeError("Run the real data loading cell first.")

rsm = float(globals().get("REAL_SMOOTH_MULT", _SMOOTH_MULT))
r_edge = float(globals().get("REAL_MU_EDGE_FRAC", 0.0))
r_q = float(globals().get("REAL_MU_Q", 1.0))

# Log-uniform mu_max target range (h^-1), independent of real-data values.
MU_MAX_LO = 0.05
MU_MAX_HI = 1.5

# Probability of broader/tail cases for extra variety.
VARIETY_TAIL_PROB = float(globals().get("VARIETY_TAIL_PROB", 0.35))

# Model mixture for synthetic generation.
SYN_MODELS = list(globals().get("SYN_MODELS", ["gompertz", "logistic", "richards"]))
SYN_MODEL_P = np.asarray(globals().get("SYN_MODEL_P", [0.45, 0.30, 0.25]), dtype=float)
if len(SYN_MODELS) != len(SYN_MODEL_P):
    raise RuntimeError("SYN_MODELS and SYN_MODEL_P must have same length.")
SYN_MODEL_P = np.clip(SYN_MODEL_P, 0.0, None)
if float(np.sum(SYN_MODEL_P)) <= 0:
    SYN_MODEL_P = np.ones(len(SYN_MODELS), dtype=float)
SYN_MODEL_P = SYN_MODEL_P / np.sum(SYN_MODEL_P)

# Force n-points and time-span independent of real-data histograms.
SYN_NPTS_RANGE = (100, 1000)
SYN_TSPAN_RANGE = (24.0, 96.0)
if len(SYN_NPTS_RANGE) != 2 or len(SYN_TSPAN_RANGE) != 2:
    raise RuntimeError("SYN_NPTS_RANGE and SYN_TSPAN_RANGE must be length-2 tuples.")
_n_lo, _n_hi = int(SYN_NPTS_RANGE[0]), int(SYN_NPTS_RANGE[1])
if _n_hi < _n_lo:
    _n_lo, _n_hi = _n_hi, _n_lo
SYN_NPTS_RANGE = (max(10, _n_lo), max(10, _n_hi))
_t_lo, _t_hi = float(SYN_TSPAN_RANGE[0]), float(SYN_TSPAN_RANGE[1])
if _t_hi < _t_lo:
    _t_lo, _t_hi = _t_hi, _t_lo
SYN_TSPAN_RANGE = (max(1.0, _t_lo), max(1.0, _t_hi))

SYN_OD_FOLD_RANGE = (0.0, 200.0)
if len(SYN_OD_FOLD_RANGE) != 2:
    raise RuntimeError("SYN_OD_FOLD_RANGE must be a length-2 tuple.")
_f_lo, _f_hi = float(SYN_OD_FOLD_RANGE[0]), float(SYN_OD_FOLD_RANGE[1])
if _f_hi < _f_lo:
    _f_lo, _f_hi = _f_hi, _f_lo
SYN_OD_FOLD_RANGE = (max(0.0, _f_lo), max(0.0, _f_hi))

# Increase synthetic test-set noise relative to real-data proxy sigma.
SYN_NOISE_MULT = float(globals().get("SYN_NOISE_MULT", 2.0))
SYN_NOISE_RANGE = tuple(globals().get("SYN_NOISE_RANGE", (1e-4, 0.80)))
if len(SYN_NOISE_RANGE) != 2:
    raise RuntimeError("SYN_NOISE_RANGE must be a length-2 tuple.")
_noise_lo, _noise_hi = float(SYN_NOISE_RANGE[0]), float(SYN_NOISE_RANGE[1])
if _noise_hi < _noise_lo:
    _noise_lo, _noise_hi = _noise_hi, _noise_lo
SYN_NOISE_RANGE = (max(1e-8, _noise_lo), max(1e-8, _noise_hi))

# One master dataset configuration (generated once here).
EVAL_N = max(500, int(globals().get("EVAL_N", 500)))
EVAL_SEED = int(globals().get("EVAL_SEED", 20260224))
EVAL_MIN_OD_INCREASE = float(globals().get("MIN_OD_INCREASE", 0.0))
EVAL_POOL_MULT = int(globals().get("EVAL_POOL_MULT", 8))
EVAL_NPTS_BINS = np.asarray(globals().get("EVAL_NPTS_BINS", [220, 400, 600, 800]), dtype=float)
EVAL_TSPAN_BINS = np.asarray(globals().get("EVAL_TSPAN_BINS", [30.0, 40.0, 52.0, 64.0, 80.0]), dtype=float)

# Stress-test injection: explicit scenarios where one method is likely to fail.
EVAL_STRESS_CASES_PER_SCENARIO = int(globals().get("EVAL_STRESS_CASES_PER_SCENARIO", 32))
EVAL_STRESS_FRACTION = float(globals().get("EVAL_STRESS_FRACTION", 0.40))
EVAL_STRESS_MAX_KEEP = int(globals().get("EVAL_STRESS_MAX_KEEP", 220))
EVAL_STRESS_TARGET = int(globals().get("EVAL_STRESS_TARGET", -1))


def _safe_quantile(arr, q, default=np.nan):
    arr = np.asarray(arr, dtype=float)
    arr = arr[np.isfinite(arr)]
    if len(arr) == 0:
        return float(default)
    return float(np.quantile(arr, q))


def _estimate_true_mu_from_curve(t, N_true, n_eval=420):
    """Approximate true mu_max = max d(ln N)/dt from noiseless curve."""
    t = np.asarray(t, dtype=float)
    N_true = np.asarray(N_true, dtype=float)
    if len(t) < 4 or np.ptp(t) <= 0:
        return np.nan

    order = np.argsort(t, kind="mergesort")
    t = t[order]
    N_true = np.clip(N_true[order], 1e-12, None)

    te = np.linspace(float(np.min(t)), float(np.max(t)), int(max(120, n_eval)))
    Ne = np.interp(te, t, N_true)
    y = np.log(np.clip(Ne, 1e-12, None))
    mu = np.gradient(y, te)
    mu = mu[np.isfinite(mu)]
    if len(mu) == 0:
        return np.nan
    return float(np.max(mu))


def _estimate_lag_fraction(t, mu):
    t = np.asarray(t, dtype=float)
    mu = np.asarray(mu, dtype=float)
    if len(t) < 3 or len(mu) != len(t):
        return 0.35
    peak = int(np.nanargmax(mu))
    if peak < 1:
        return 0.20
    mu_peak = float(np.nanmax(mu))
    if (not np.isfinite(mu_peak)) or (mu_peak <= 0):
        return 0.35
    thr = 0.25 * mu_peak
    idx = np.where(mu[: peak + 1] >= thr)[0]
    if len(idx) == 0:
        return 0.35
    span = max(float(np.ptp(t)), 1e-8)
    return float(np.clip((t[int(idx[0])] - float(t[0])) / span, 0.02, 0.95))


def _describe_real_curve(rec):
    t = np.asarray(rec["t"], dtype=float)
    N = np.asarray(rec["N"], dtype=float)
    if len(t) < 10 or np.ptp(t) <= 0:
        return None
    y = np.log(np.clip(N, 1e-12, None))
    sigma = _robust_sigma_from_second_diff(t, y, trim_q=0.80)
    try:
        sp, s_opt, _ = auto_smooth_spline(t, N, smooth_mult=rsm)
    except Exception:
        return None
    t_dense = np.linspace(float(np.min(t)), float(np.max(t)), 320)
    mu = np.asarray(sp.derivative()(t_dense), dtype=float)
    mu_hat = float(np.nanmax(mu)) if np.any(np.isfinite(mu)) else np.nan
    lag_frac = _estimate_lag_fraction(t_dense, mu)
    od_p05 = float(np.percentile(N, 5))
    od_p95 = float(np.percentile(N, 95))
    od_fold = float(od_p95 / max(od_p05, 1e-8))
    return {
        "n_pts": int(len(t)),
        "t_span": float(np.ptp(t)),
        "od_p05": od_p05,
        "od_p95": od_p95,
        "od_fold": od_fold,
        "sigma": float(max(1e-4, sigma)),
        "mu_hat": float(max(1e-4, mu_hat)) if np.isfinite(mu_hat) else np.nan,
        "lag_frac": lag_frac,
        "t_template": np.asarray(t, dtype=float),
        "rec": rec,
        "s_opt": float(s_opt),
    }


real_desc = []
for rec in real_curves_auto:
    d = _describe_real_curve(rec)
    if d is not None and np.isfinite(d["mu_hat"]) and d["od_fold"] > 1.02:
        real_desc.append(d)

if len(real_desc) < 10:
    raise RuntimeError("Not enough describable real curves to build a synthetic profile.")

arr_n = np.asarray([d["n_pts"] for d in real_desc], dtype=float)
arr_span = np.asarray([d["t_span"] for d in real_desc], dtype=float)
arr_sigma = np.asarray([d["sigma"] for d in real_desc], dtype=float)
arr_mu = np.asarray([d["mu_hat"] for d in real_desc], dtype=float)
arr_lag = np.asarray([d["lag_frac"] for d in real_desc], dtype=float)
arr_fold = np.asarray([d["od_fold"] for d in real_desc], dtype=float)
arr_od0 = np.asarray([d["od_p05"] for d in real_desc], dtype=float)

print("Real-data profile used for synthetic generation")
print(f"  curves profiled: {len(real_desc)}")
print(f"  n points: median={np.median(arr_n):.1f}, q10={_safe_quantile(arr_n,0.10):.1f}, q90={_safe_quantile(arr_n,0.90):.1f}")
print(f"  t span (h): median={np.median(arr_span):.2f}, q10={_safe_quantile(arr_span,0.10):.2f}, q90={_safe_quantile(arr_span,0.90):.2f}")
print(f"  OD fold (p95/p05): median={np.median(arr_fold):.2f}, q10={_safe_quantile(arr_fold,0.10):.2f}, q90={_safe_quantile(arr_fold,0.90):.2f}")
print(f"  sigma proxy: median={np.median(arr_sigma):.4f}, q10={_safe_quantile(arr_sigma,0.10):.4f}, q90={_safe_quantile(arr_sigma,0.90):.4f}")
print(f"  synthetic test-set noise: sigma_proxy * {SYN_NOISE_MULT:.2f}, clipped to [{SYN_NOISE_RANGE[0]:.4f}, {SYN_NOISE_RANGE[1]:.4f}]")
print(f"  mu_max proxy (real, informational): median={np.median(arr_mu):.3f}, q10={_safe_quantile(arr_mu,0.10):.3f}, q90={_safe_quantile(arr_mu,0.90):.3f}")
print(f"  mu_max sampling target: log-uniform [{MU_MAX_LO}, {MU_MAX_HI}] h^-1")
print(f"  tail-case probability (variety boost): {VARIETY_TAIL_PROB:.2f}")
print("  synthetic model mix: " + ", ".join([f"{m}:{p:.2f}" for m, p in zip(SYN_MODELS, SYN_MODEL_P)]))
print(
    f"  synthetic n_points range (uniform): [{SYN_NPTS_RANGE[0]}, {SYN_NPTS_RANGE[1]}], "
    f"t_span range (uniform h): [{SYN_TSPAN_RANGE[0]:.1f}, {SYN_TSPAN_RANGE[1]:.1f}], "
    f"od_fold range (uniform): [{SYN_OD_FOLD_RANGE[0]:.1f}, {SYN_OD_FOLD_RANGE[1]:.1f}]"
)


def _sample_real_like_curve(rng):
    # Template shape from real data, but force n-points and t-span to uniform ranges.
    base = real_desc[int(rng.integers(0, len(real_desc)))]
    t_base = np.asarray(base["t_template"], dtype=float)
    t0 = float(np.min(t_base))
    t_norm_base = (t_base - t0) / max(float(np.ptp(t_base)), 1e-8)

    tail_case = bool(rng.random() < VARIETY_TAIL_PROB)

    n0_jitter = 0.30 if tail_case else 0.18
    lag_jitter = 0.14 if tail_case else 0.06

    n_target = int(rng.integers(SYN_NPTS_RANGE[0], SYN_NPTS_RANGE[1] + 1))
    n_target = int(np.clip(n_target, 100, 1000))
    t_span = float(rng.uniform(SYN_TSPAN_RANGE[0], SYN_TSPAN_RANGE[1]))

    # Resample template timing shape to chosen n_target via quantile interpolation.
    q_old = np.linspace(0.0, 1.0, len(t_norm_base))
    q_new = np.linspace(0.0, 1.0, n_target)
    t_norm = np.interp(q_new, q_old, t_norm_base)

    # Add mild warp and tiny internal jitter for variety while preserving monotonicity.
    warp = float(rng.uniform(0.75, 1.35) if tail_case else rng.uniform(0.90, 1.12))
    t_norm = np.clip(t_norm ** warp, 0.0, 1.0)
    if n_target > 6:
        jitter_sd = 0.010 if tail_case else 0.004
        jitter = rng.normal(0.0, jitter_sd, size=n_target)
        jitter[0] = 0.0
        jitter[-1] = 0.0
        t_norm = np.clip(t_norm + jitter, 0.0, 1.0)
        t_norm = np.sort(t_norm)
        t_norm[0] = 0.0
        t_norm[-1] = 1.0

    t = t_norm * t_span
    t = np.maximum.accumulate(t)
    if t[-1] > t[0]:
        t = (t - t[0]) / (t[-1] - t[0]) * t_span

    N0 = float(np.clip(rng.choice(arr_od0) * rng.lognormal(0.0, n0_jitter), 1e-4, 3.0))
    fold = float(rng.uniform(SYN_OD_FOLD_RANGE[0], SYN_OD_FOLD_RANGE[1]))
    Nmax = float(max(1.01 * N0, N0 * max(fold, 0.0)))

    lag_frac = float(np.clip(rng.choice(arr_lag) + rng.normal(0.0, lag_jitter), 0.01, 0.96))
    lag = float(lag_frac * t_span)

    # Use a noisier test set than real-data sigma proxy.
    noise_sigma = float(rng.choice(arr_sigma))
    if (not np.isfinite(noise_sigma)) or (noise_sigma <= 0):
        noise_sigma = float(np.nanmedian(arr_sigma)) if np.any(np.isfinite(arr_sigma)) else 1e-4
    noise_sigma = float(np.clip(noise_sigma * SYN_NOISE_MULT, SYN_NOISE_RANGE[0], SYN_NOISE_RANGE[1]))

    mu_target = float(np.exp(rng.uniform(np.log(MU_MAX_LO), np.log(MU_MAX_HI))))
    model_name = str(rng.choice(SYN_MODELS, p=SYN_MODEL_P))
    richards_nu = float(np.clip(rng.lognormal(0.0, 0.35), 0.35, 3.5)) if model_name == "richards" else 1.0

    N_true, N_obs = make_growth_curve(
        t,
        N0=N0,
        Nmax=Nmax,
        mu_max=mu_target,
        lag=lag,
        noise_sigma=noise_sigma,
        seed=int(rng.integers(0, 1_000_000_000)),
        model=model_name,
        richards_nu=richards_nu,
    )

    mu_true = _estimate_true_mu_from_curve(t, N_true)
    if (not np.isfinite(mu_true)) or (mu_true <= 0):
        mu_true = float(max(1e-6, mu_target))

    return t, N_true, N_obs, mu_true, noise_sigma, N0, Nmax, model_name


def _bin_index(v, edges):
    return int(np.digitize([float(v)], edges)[0])


def _uniform_from_range(rng, lo_hi):
    lo = float(lo_hi[0])
    hi = float(lo_hi[1])
    if hi < lo:
        lo, hi = hi, lo
    return float(lo if abs(hi - lo) < 1e-12 else rng.uniform(lo, hi))


def _make_eval_record(t, N_true, N_obs, mu_true, noise_sigma, model_name, curve_id,
                      is_stress=False, stress_tag="", target_fail="none"):
    t = np.asarray(t, dtype=float)
    N_true = np.asarray(N_true, dtype=float)
    N_obs = np.asarray(N_obs, dtype=float)

    if len(t) < 10 or np.ptp(t) <= 0:
        return None

    od_inc = float(np.nanmax(N_obs) - np.nanmin(N_obs))
    if (not np.isfinite(od_inc)) or (od_inc < EVAL_MIN_OD_INCREASE):
        return None

    mu_true = float(mu_true)
    if (not np.isfinite(mu_true)) or (mu_true <= 0):
        mu_true = _estimate_true_mu_from_curve(t, N_true)
    if (not np.isfinite(mu_true)) or (mu_true <= 0):
        return None

    return {
        "curve_id": str(curve_id),
        "t": np.asarray(t, dtype=float),
        "N_true": np.asarray(N_true, dtype=float),
        "N_obs": np.asarray(N_obs, dtype=float),
        "mu_true": float(max(1e-6, mu_true)),
        "noise_sigma": float(max(1e-8, noise_sigma)),
        "t_span": float(np.ptp(t)),
        "n_pts": int(len(t)),
        "model": str(model_name),
        "od_fold": float(np.percentile(N_obs, 95) / max(np.percentile(N_obs, 5), 1e-8)),
        "is_stress": bool(is_stress),
        "stress_tag": str(stress_tag),
        "target_fail": str(target_fail),
    }


def _inject_sparse_spikes(N_obs, rng, spike_prob=0.0, spike_mult_range=(1.0, 1.0)):
    N_obs = np.asarray(N_obs, dtype=float).copy()
    p = float(max(0.0, spike_prob))
    if p <= 0 or len(N_obs) < 5:
        return np.clip(N_obs, 1e-10, None)

    mask = rng.random(len(N_obs)) < p
    mask[0] = False
    mask[-1] = False
    if not np.any(mask):
        return np.clip(N_obs, 1e-10, None)

    lo = float(max(1.01, min(spike_mult_range)))
    hi = float(max(lo, max(spike_mult_range)))
    mags = np.exp(rng.uniform(np.log(lo), np.log(hi), size=int(np.sum(mask))))
    N_obs[mask] = N_obs[mask] * mags
    return np.clip(N_obs, 1e-10, None)


def _generate_stress_pool(rng, n_per_scenario):
    n_per_scenario = int(max(0, n_per_scenario))
    if n_per_scenario <= 0:
        return []

    scenarios = [
        {
            "tag": "baseline_high_noise_spikes",
            "target_fail": "baseline",
            "model": "gompertz",
            "n_pts_range": (180, 420),
            "t_span_range": (24.0, 52.0),
            "n0_range": (0.01, 0.06),
            "fold_range": (12.0, 80.0),
            "mu_range": (0.20, 0.55),
            "lag_frac_range": (0.08, 0.35),
            "noise_range": (0.45, 0.80),
            "warp_range": (0.55, 1.15),
            "spike_prob": 0.05,
            "spike_mult_range": (2.5, 5.5),
            "richards_nu_range": (1.0, 1.0),
        },
        {
            "tag": "baseline_dense_noisy",
            "target_fail": "baseline",
            "model": "logistic",
            "n_pts_range": (650, 1000),
            "t_span_range": (30.0, 80.0),
            "n0_range": (0.01, 0.08),
            "fold_range": (8.0, 60.0),
            "mu_range": (0.12, 0.40),
            "lag_frac_range": (0.15, 0.45),
            "noise_range": (0.30, 0.70),
            "warp_range": (0.70, 1.50),
            "spike_prob": 0.02,
            "spike_mult_range": (2.0, 4.0),
            "richards_nu_range": (1.0, 1.0),
        },
        {
            "tag": "baseline_sparse_extreme_noise",
            "target_fail": "baseline",
            "model": "logistic",
            "n_pts_range": (100, 220),
            "t_span_range": (24.0, 60.0),
            "n0_range": (0.005, 0.05),
            "fold_range": (6.0, 40.0),
            "mu_range": (0.08, 0.30),
            "lag_frac_range": (0.10, 0.55),
            "noise_range": (0.55, 0.80),
            "warp_range": (0.45, 1.25),
            "spike_prob": 0.10,
            "spike_mult_range": (3.0, 8.0),
            "richards_nu_range": (1.0, 1.0),
        },
        {
            "tag": "baseline_jagged_bursts",
            "target_fail": "baseline",
            "model": "gompertz",
            "n_pts_range": (220, 520),
            "t_span_range": (28.0, 90.0),
            "n0_range": (0.008, 0.07),
            "fold_range": (10.0, 70.0),
            "mu_range": (0.15, 0.60),
            "lag_frac_range": (0.05, 0.42),
            "noise_range": (0.40, 0.80),
            "warp_range": (0.50, 1.60),
            "spike_prob": 0.12,
            "spike_mult_range": (4.0, 10.0),
            "richards_nu_range": (1.0, 1.0),
        },
        {
            "tag": "auto_sharp_transition",
            "target_fail": "auto",
            "model": "richards",
            "n_pts_range": (350, 900),
            "t_span_range": (36.0, 96.0),
            "n0_range": (0.02, 0.10),
            "fold_range": (30.0, 180.0),
            "mu_range": (0.90, 1.60),
            "lag_frac_range": (0.35, 0.70),
            "noise_range": (0.02, 0.08),
            "warp_range": (0.90, 2.00),
            "spike_prob": 0.0,
            "spike_mult_range": (1.0, 1.0),
            "richards_nu_range": (0.35, 0.70),
        },
        {
            "tag": "auto_late_fast_edge",
            "target_fail": "auto",
            "model": "gompertz",
            "n_pts_range": (250, 700),
            "t_span_range": (28.0, 84.0),
            "n0_range": (0.02, 0.10),
            "fold_range": (20.0, 120.0),
            "mu_range": (0.80, 1.50),
            "lag_frac_range": (0.72, 0.93),
            "noise_range": (0.02, 0.08),
            "warp_range": (1.10, 2.20),
            "spike_prob": 0.0,
            "spike_mult_range": (1.0, 1.0),
            "richards_nu_range": (1.0, 1.0),
        },
        {
            "tag": "auto_ultra_sharp_richards",
            "target_fail": "auto",
            "model": "richards",
            "n_pts_range": (500, 1000),
            "t_span_range": (32.0, 96.0),
            "n0_range": (0.015, 0.08),
            "fold_range": (50.0, 220.0),
            "mu_range": (1.20, 2.40),
            "lag_frac_range": (0.45, 0.82),
            "noise_range": (0.01, 0.05),
            "warp_range": (1.40, 3.20),
            "spike_prob": 0.0,
            "spike_mult_range": (1.0, 1.0),
            "richards_nu_range": (0.20, 0.45),
        },
        {
            "tag": "auto_terminal_takeoff",
            "target_fail": "auto",
            "model": "logistic",
            "n_pts_range": (280, 900),
            "t_span_range": (24.0, 72.0),
            "n0_range": (0.02, 0.10),
            "fold_range": (15.0, 110.0),
            "mu_range": (1.00, 2.20),
            "lag_frac_range": (0.88, 0.97),
            "noise_range": (0.01, 0.06),
            "warp_range": (1.80, 3.50),
            "spike_prob": 0.0,
            "spike_mult_range": (1.0, 1.0),
            "richards_nu_range": (1.0, 1.0),
        },
    ]

    stress_pool = []
    for sc in scenarios:
        for j in range(n_per_scenario):
            n_pts = int(np.clip(int(round(_uniform_from_range(rng, sc["n_pts_range"]))), 100, 1000))
            t_span = float(np.clip(_uniform_from_range(rng, sc["t_span_range"]), SYN_TSPAN_RANGE[0], SYN_TSPAN_RANGE[1]))

            q = np.linspace(0.0, 1.0, n_pts)
            warp = _uniform_from_range(rng, sc["warp_range"])
            t = np.power(q, warp) * t_span

            if n_pts > 8:
                jitter_sd = 0.002 if sc["target_fail"] == "auto" else 0.004
                jitter = rng.normal(0.0, jitter_sd, size=n_pts)
                jitter[0] = 0.0
                jitter[-1] = 0.0
                t = np.clip(t + jitter * t_span, 0.0, t_span)
                t = np.sort(t)
                t[0] = 0.0
                t[-1] = t_span

            N0 = _uniform_from_range(rng, sc["n0_range"])
            fold = _uniform_from_range(rng, sc["fold_range"])
            Nmax = float(max(1.01 * N0, N0 * fold))
            mu_max = _uniform_from_range(rng, sc["mu_range"])
            lag_frac = _uniform_from_range(rng, sc["lag_frac_range"])
            lag = float(np.clip(lag_frac * t_span, 0.01 * t_span, 0.97 * t_span))
            noise_sigma = float(np.clip(
                _uniform_from_range(rng, sc["noise_range"]),
                SYN_NOISE_RANGE[0],
                SYN_NOISE_RANGE[1],
            ))

            model_name = str(sc["model"])
            richards_nu = _uniform_from_range(rng, sc["richards_nu_range"]) if model_name == "richards" else 1.0

            N_true, N_obs = make_growth_curve(
                t,
                N0=N0,
                Nmax=Nmax,
                mu_max=mu_max,
                lag=lag,
                noise_sigma=noise_sigma,
                seed=int(rng.integers(0, 1_000_000_000)),
                model=model_name,
                richards_nu=richards_nu,
            )

            N_obs = _inject_sparse_spikes(
                N_obs,
                rng,
                spike_prob=float(sc.get("spike_prob", 0.0)),
                spike_mult_range=tuple(sc.get("spike_mult_range", (1.0, 1.0))),
            )

            mu_true = _estimate_true_mu_from_curve(t, N_true)
            rec = _make_eval_record(
                t,
                N_true,
                N_obs,
                mu_true,
                noise_sigma,
                model_name,
                curve_id=f"stress_{sc['tag']}_{j:04d}",
                is_stress=True,
                stress_tag=sc["tag"],
                target_fail=sc["target_fail"],
            )
            if rec is not None:
                stress_pool.append(rec)

    return stress_pool


# Build ONE master evaluation dataset (eval_curves).
rng_eval = np.random.default_rng(EVAL_SEED)
cal_pool = []
attempts = 0
target_pool = EVAL_N * max(2, EVAL_POOL_MULT)
# Regenerate dropped curves until target pool size is reached.
max_attempts = EVAL_N * max(200, EVAL_POOL_MULT * 80)

while len(cal_pool) < target_pool:
    attempts += 1
    if attempts > max_attempts:
        raise RuntimeError(
            f"Could not build enough usable curves: {len(cal_pool)} / {target_pool} after {attempts} attempts."
        )

    t, N_true, N_obs, mu_true, noise_sigma, N0, Nmax, model_name = _sample_real_like_curve(rng_eval)
    rec = _make_eval_record(
        t,
        N_true,
        N_obs,
        mu_true,
        noise_sigma,
        model_name,
        curve_id=f"random_{attempts:06d}",
        is_stress=False,
        stress_tag="",
        target_fail="none",
    )
    if rec is None:
        continue
    cal_pool.append(rec)

# Add explicit stress-test scenarios likely to expose baseline/auto failure modes.
stress_pool = _generate_stress_pool(rng_eval, EVAL_STRESS_CASES_PER_SCENARIO)
cal_pool.extend(stress_pool)

stress_counts_pool = {}
stress_target_counts_pool = {}
for r in stress_pool:
    tag = str(r.get("stress_tag", "stress"))
    stress_counts_pool[tag] = stress_counts_pool.get(tag, 0) + 1
    tgt = str(r.get("target_fail", "unknown"))
    stress_target_counts_pool[tgt] = stress_target_counts_pool.get(tgt, 0) + 1

if len(cal_pool) < max(250, int(0.5 * EVAL_N)):
    raise RuntimeError(f"Only {len(cal_pool)} usable synthetic curves generated (target={EVAL_N}).")

strata = {}
for rec in cal_pool:
    k = (_bin_index(rec["n_pts"], EVAL_NPTS_BINS), _bin_index(rec["t_span"], EVAL_TSPAN_BINS))
    strata.setdefault(k, []).append(rec)

for k in strata:
    rng_eval.shuffle(strata[k])

all_keys = sorted(strata.keys())
if len(all_keys) == 0:
    raise RuntimeError("No strata formed for evaluation-set selection.")

eval_curves = []
used_by_key = {k: 0 for k in all_keys}
while len(eval_curves) < EVAL_N:
    progressed = False
    for k in all_keys:
        i = used_by_key[k]
        bucket = strata[k]
        if i < len(bucket):
            eval_curves.append(bucket[i])
            used_by_key[k] += 1
            progressed = True
            if len(eval_curves) >= EVAL_N:
                break
    if not progressed:
        break

# Ensure stress scenarios are present in final eval_curves at a useful fraction.
stress_target_keep = EVAL_STRESS_TARGET
if stress_target_keep < 0:
    stress_target_keep = int(round(EVAL_STRESS_FRACTION * EVAL_N))
stress_target_keep = int(np.clip(
    stress_target_keep,
    0,
    min(EVAL_STRESS_MAX_KEEP, EVAL_N, len(stress_pool)),
))

if stress_target_keep > 0 and len(eval_curves) > 0:
    selected_ids = {str(r.get("curve_id", "")) for r in eval_curves}
    stress_now = [i for i, r in enumerate(eval_curves) if bool(r.get("is_stress", False))]
    need = max(0, stress_target_keep - len(stress_now))

    if need > 0:
        candidate_idxs = [i for i, r in enumerate(eval_curves) if not bool(r.get("is_stress", False))]
        rng_eval.shuffle(candidate_idxs)

        extra_stress = [r for r in stress_pool if str(r.get("curve_id", "")) not in selected_ids]
        rng_eval.shuffle(extra_stress)

        for rep_idx, new_rec in zip(candidate_idxs, extra_stress[:need]):
            eval_curves[rep_idx] = new_rec
            selected_ids.add(str(new_rec.get("curve_id", "")))

# If stratified selection undershoots target, regenerate replacement curves until exactly EVAL_N.
fill_attempts = 0
max_fill_attempts = EVAL_N * 200
while len(eval_curves) < EVAL_N:
    fill_attempts += 1
    attempts += 1
    if fill_attempts > max_fill_attempts:
        raise RuntimeError(
            f"Could not refill eval_curves to {EVAL_N}; current={len(eval_curves)} after {fill_attempts} refill attempts."
        )

    t, N_true, N_obs, mu_true, noise_sigma, N0, Nmax, model_name = _sample_real_like_curve(rng_eval)
    rec = _make_eval_record(
        t,
        N_true,
        N_obs,
        mu_true,
        noise_sigma,
        model_name,
        curve_id=f"refill_{attempts:06d}",
        is_stress=False,
        stress_tag="",
        target_fail="none",
    )
    if rec is None:
        continue
    eval_curves.append(rec)

if len(eval_curves) > EVAL_N:
    eval_curves = eval_curves[:EVAL_N]

if len(eval_curves) != EVAL_N:
    raise RuntimeError(f"Evaluation set size mismatch: got {len(eval_curves)}, expected {EVAL_N}.")

arr_mu_eval = np.asarray([r["mu_true"] for r in eval_curves], dtype=float)
arr_noise_eval = np.asarray([r["noise_sigma"] for r in eval_curves], dtype=float)
arr_span_eval = np.asarray([r["t_span"] for r in eval_curves], dtype=float)
arr_npts_eval = np.asarray([r["n_pts"] for r in eval_curves], dtype=float)
arr_fold_eval = np.asarray([r["od_fold"] for r in eval_curves], dtype=float)

if np.min(arr_npts_eval) < 100 or np.max(arr_npts_eval) > 1000:
    raise RuntimeError(
        f"Generated n_pts out of bounds: [{np.min(arr_npts_eval):.0f}, {np.max(arr_npts_eval):.0f}] (expected 100..1000)."
    )

model_counts_eval = {}
stress_counts_eval = {}
stress_target_counts_eval = {}
for r in eval_curves:
    model_counts_eval[r["model"]] = model_counts_eval.get(r["model"], 0) + 1
    if bool(r.get("is_stress", False)):
        tag = str(r.get("stress_tag", "stress"))
        stress_counts_eval[tag] = stress_counts_eval.get(tag, 0) + 1
        tgt = str(r.get("target_fail", "unknown"))
        stress_target_counts_eval[tgt] = stress_target_counts_eval.get(tgt, 0) + 1

print(f"Evaluation set ready: n={len(eval_curves)} (attempts={attempts}, pool={len(cal_pool)})")
print(
    "  mu_true range: "
    f"q05={np.quantile(arr_mu_eval,0.05):.3f}, med={np.median(arr_mu_eval):.3f}, q95={np.quantile(arr_mu_eval,0.95):.3f}"
)
print(
    "  noise_sigma range: "
    f"q05={np.quantile(arr_noise_eval,0.05):.4f}, med={np.median(arr_noise_eval):.4f}, q95={np.quantile(arr_noise_eval,0.95):.4f}"
)
print(
    "  n_pts/t_span/od_fold medians: "
    f"n_pts={np.median(arr_npts_eval):.1f}, t_span={np.median(arr_span_eval):.2f}, od_fold={np.median(arr_fold_eval):.2f}"
)
print(
    "  n_pts/t_span ranges: "
    f"n_pts=[{np.min(arr_npts_eval):.0f}, {np.max(arr_npts_eval):.0f}], "
    f"t_span=[{np.min(arr_span_eval):.2f}, {np.max(arr_span_eval):.2f}] h"
)
print("  model counts:", model_counts_eval)
print(f"  stress pool generated: {len(stress_pool)}")
if stress_counts_pool:
    print("  stress pool by scenario:", stress_counts_pool)
if stress_target_counts_pool:
    print("  stress pool by expected failure mode:", stress_target_counts_pool)
print(
    f"  stress curves in eval set: {sum(stress_counts_eval.values())}/{len(eval_curves)} "
    f"(target >= {stress_target_keep})"
)
if stress_counts_eval:
    print("  stress eval by scenario:", stress_counts_eval)
if stress_target_counts_eval:
    print("  stress eval by expected failure mode:", stress_target_counts_eval)
print("  strata counts (npts_bin, tspan_bin -> n):")
for k in all_keys:
    print(f"    {k}: {used_by_key[k]}")


# Distribution comparison: real vs synthetic (use full eval_curves).
arr_syn_n = np.asarray([len(s["t"]) for s in eval_curves], dtype=float)
arr_syn_span = np.asarray([s["t_span"] for s in eval_curves], dtype=float)
arr_syn_sigma = np.asarray([s["noise_sigma"] for s in eval_curves], dtype=float)
arr_syn_fold = np.asarray([s["od_fold"] for s in eval_curves], dtype=float)

fig_dist = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=["n points", "time span (h)", "noise proxy sigma", "OD fold (p95/p05)"],
    horizontal_spacing=0.12,
    vertical_spacing=0.16,
)
for row, col, real_arr, syn_arr in [
    (1, 1, arr_n, arr_syn_n),
    (1, 2, arr_span, arr_syn_span),
    (2, 1, arr_sigma, arr_syn_sigma),
    (2, 2, arr_fold, arr_syn_fold),
]:
    sl = (row == 1 and col == 1)
    fig_dist.add_trace(
        go.Histogram(x=real_arr, nbinsx=25, name="Real",
                     opacity=0.60, marker=dict(color="#8a8a8a"), showlegend=sl),
        row=row,
        col=col,
    )
    fig_dist.add_trace(
        go.Histogram(x=syn_arr, nbinsx=25, name="Synthetic",
                     opacity=0.60, marker=dict(color=COLORS["auto"]), showlegend=sl),
        row=row,
        col=col,
    )
fig_dist.update_layout(
    title="Real vs synthetic descriptor distributions",
    barmode="overlay",
    width=1080,
    height=700,
    template=PLOTLY_TEMPLATE,
)
fig_dist.show(config=PLOTLY_CONFIG)


# Overlay plots: sample a subset from the same eval_curves (no second data generation).
rng_overlay = np.random.default_rng(EVAL_SEED + 1)
N_SYN_OVERLAY = int(globals().get("N_SYN_OVERLAY", 12))
N_SYN_OVERLAY = int(np.clip(N_SYN_OVERLAY, 1, len(eval_curves)))
overlay_idx = np.sort(rng_overlay.choice(len(eval_curves), size=N_SYN_OVERLAY, replace=False))

def _adaptive_baseline_fit_overlay(t, N):
    t = np.asarray(t, dtype=float)
    N = np.asarray(N, dtype=float)
    y = np.log(np.clip(N, 1e-12, None))
    n = max(len(t), 2)

    sigma_i = _robust_sigma_from_second_diff(t, y, trim_q=0.80)
    if (not np.isfinite(sigma_i)) or (sigma_i <= 0):
        sigma_i = 1e-4

    s_min = 0.01
    s_max = max(s_min * 10, 0.8 * float(n) * float(np.var(y)))
    s_i = float(np.clip(float(n) * float(sigma_i) ** 2, s_min, s_max))
    return fixed_s_spline(t, N, s=s_i), s_i, float(sigma_i)


synthetic_selected = []
for idx in overlay_idx:
    rec = eval_curves[int(idx)]
    t = rec["t"]
    N_obs = rec["N_obs"]
    N_true = rec["N_true"]
    try:
        sp_auto, s_opt, sigma_est = auto_smooth_spline(t, N_obs, smooth_mult=rsm)
        sp_base, s_base, sigma_base = _adaptive_baseline_fit_overlay(t, N_obs)
    except Exception:
        continue

    t_dense = np.linspace(float(np.min(t)), float(np.max(t)), 400)
    mu_auto = _mu_max_from_spline(sp_auto, t, edge_frac=r_edge, q=r_q, n_eval=400)
    mu_base = _mu_max_from_spline(sp_base, t, edge_frac=r_edge, q=r_q, n_eval=400)

    synthetic_selected.append({
        "t": t,
        "N_obs": N_obs,
        "N_true": N_true,
        "sp_auto": sp_auto,
        "sp_base": sp_base,
        "t_dense": t_dense,
        "mu_true": float(rec["mu_true"]),
        "mu_auto": mu_auto,
        "mu_base": mu_base,
        "s_opt": s_opt,
        "sigma": float(rec["noise_sigma"]),
        "s_base": float(s_base),
        "model": rec["model"],
        "t_span": float(rec["t_span"]),
        "od_fold": float(rec["od_fold"]),
    })

if not synthetic_selected:
    raise RuntimeError("Could not select usable curves for overlay plots from eval_curves.")

n_plot = len(synthetic_selected)
n_cols = 4
n_rows = int(np.ceil(n_plot / n_cols))
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    subplot_titles=[" "] * (n_rows * n_cols),
    horizontal_spacing=0.05,
    vertical_spacing=0.08,
)

for i, item in enumerate(synthetic_selected):
    r, c = _row_col(i, n_cols)
    t = item["t"]
    N_obs = item["N_obs"]
    N_true = item["N_true"]
    t_dense = item["t_dense"]
    N_fit_auto = np.exp(item["sp_auto"](t_dense))
    N_fit_base = np.exp(item["sp_base"](t_dense))
    sl = (i == 0)

    fig.add_trace(go.Scatter(
        x=t, y=N_obs, mode="markers",
        marker=dict(size=5, color=COLORS["data"], opacity=0.65),
        name="Noisy data", showlegend=sl), row=r, col=c)

    fig.add_trace(go.Scatter(
        x=t_dense, y=np.interp(t_dense, t, N_true), mode="lines",
        line=dict(width=1.4, color=COLORS["true"], dash="dash"),
        name="True", showlegend=sl), row=r, col=c)

    fig.add_trace(go.Scatter(
        x=t_dense, y=N_fit_base, mode="lines",
        line=dict(width=2.0, color=COLORS["fixed"]),
        name="Baseline (adaptive s_i)", showlegend=sl), row=r, col=c)

    fig.add_trace(go.Scatter(
        x=t_dense, y=N_fit_auto, mode="lines",
        line=dict(width=2.0, color=COLORS["auto"]),
        name="Auto spline", showlegend=sl), row=r, col=c)

    fig.update_xaxes(title_text="Time (h)", row=r, col=c)
    fig.update_yaxes(title_text="OD", row=r, col=c)

    subtitle = (
        f"model={item['model']} | mu_true={item['mu_true']:.3f}<br>"
        f"baseline={item['mu_base']:.3f}, auto={item['mu_auto']:.3f}"
    )
    if i < len(fig.layout.annotations):
        fig.layout.annotations[i].text = subtitle

for j in range(n_plot, n_rows * n_cols):
    r, c = _row_col(j, n_cols)
    fig.update_xaxes(visible=False, row=r, col=c)
    fig.update_yaxes(visible=False, row=r, col=c)

fig.update_layout(
    title=(
        "Synthetic overlays from eval_curves: baseline (adaptive) vs auto spline<br>"
        f"smooth_mult={rsm:.3f}, mu_edge={r_edge:.3f}, mu_q={r_q:.3f}"
    ),
    width=1450,
    height=max(560, 280 * n_rows),
    template=PLOTLY_TEMPLATE,
)
fig.show(config=PLOTLY_CONFIG)
print(f"Plotted {n_plot} curves from eval_curves (single shared dataset).")


Real-data profile used for synthetic generation
  curves profiled: 208
  n points: median=281.0, q10=105.0, q90=694.0
  t span (h): median=138.60, q10=20.80, q90=138.98
  OD fold (p95/p05): median=3.27, q10=1.03, q90=19.19
  sigma proxy: median=0.0012, q10=0.0001, q90=0.0043
  synthetic test-set noise: sigma_proxy * 5.00, clipped to [0.0001, 0.8000]
  mu_max proxy (real, informational): median=0.231, q10=0.005, q90=6.392
  mu_max sampling target: log-uniform [0.05, 1.5] h^-1
  tail-case probability (variety boost): 0.35
  synthetic model mix: gompertz:0.45, logistic:0.30, richards:0.25
  synthetic n_points range (uniform): [100, 1000], t_span range (uniform h): [24.0, 96.0], od_fold range (uniform): [0.0, 200.0]
Evaluation set ready: n=500 (attempts=4057, pool=4192)
  mu_true range: q05=0.071, med=0.329, q95=1.652
  noise_sigma range: q05=0.0005, med=0.0100, q95=0.6632
  n_pts/t_span/od_fold medians: n_pts=514.0, t_span=51.64, od_fold=54.08
  n_pts/t_span ranges: n_pts=[100, 996], t_sp

Plotted 12 curves from eval_curves (single shared dataset).


In [244]:
# ---------------------------------------------------------------------------
# Use pre-generated eval_curves and define helper evaluator (no re-generation)
# ---------------------------------------------------------------------------

if "eval_curves" not in globals() or len(eval_curves) == 0:
    raise RuntimeError("Run the synthetic data generation cell first (it builds eval_curves).")

arr_mu = np.asarray([r["mu_true"] for r in eval_curves], dtype=float)
arr_noise = np.asarray([r["noise_sigma"] for r in eval_curves], dtype=float)
arr_span = np.asarray([r["t_span"] for r in eval_curves], dtype=float)
arr_npts = np.asarray([r["n_pts"] for r in eval_curves], dtype=float)
arr_fold = np.asarray([r["od_fold"] for r in eval_curves], dtype=float)

model_counts = {}
for r in eval_curves:
    model_counts[r["model"]] = model_counts.get(r["model"], 0) + 1

print(f"Using shared evaluation set: n={len(eval_curves)}")
print(
    "  mu_true range: "
    f"q05={np.quantile(arr_mu,0.05):.3f}, med={np.median(arr_mu):.3f}, q95={np.quantile(arr_mu,0.95):.3f}"
)
print(
    "  noise_sigma range: "
    f"q05={np.quantile(arr_noise,0.05):.4f}, med={np.median(arr_noise):.4f}, q95={np.quantile(arr_noise,0.95):.4f}"
)
print(
    "  n_pts/t_span/od_fold medians: "
    f"n_pts={np.median(arr_npts):.1f}, t_span={np.median(arr_span):.2f}, od_fold={np.median(arr_fold):.2f}"
)
print(
    "  n_pts/t_span ranges: "
    f"n_pts=[{np.min(arr_npts):.0f}, {np.max(arr_npts):.0f}], "
    f"t_span=[{np.min(arr_span):.2f}, {np.max(arr_span):.2f}] h"
)
print("  model counts:", model_counts)


def _eval_config(curves, smooth_mult, edge_frac, mu_q, fit_fn=None):
    """
    Evaluate mu_max estimation accuracy on a set of curves.

    fit_fn: optional callable(t, N) -> spline. If None, uses auto_smooth_spline
            with the given smooth_mult.
    """
    errs = []
    fail = 0
    mu_true_list, mu_hat_list = [], []
    for rec in curves:
        t = rec["t"]
        N_obs = rec["N_obs"]
        mu_true = rec["mu_true"]
        try:
            if fit_fn is not None:
                sp = fit_fn(t, N_obs)
            else:
                sp, _, _ = auto_smooth_spline(t, N_obs, smooth_mult=smooth_mult)
            mu_hat = _mu_max_from_spline(sp, t, edge_frac=edge_frac, q=mu_q, n_eval=420)
        except Exception:
            fail += 1
            continue
        if (not np.isfinite(mu_hat)) or (mu_hat <= 0):
            fail += 1
            continue
        errs.append(float(abs(mu_hat - mu_true) / (mu_true + 1e-8)))
        mu_true_list.append(float(mu_true))
        mu_hat_list.append(float(mu_hat))

    errs = np.asarray(errs, dtype=float)
    if len(errs) == 0:
        return {
            "score": np.inf,
            "mean": np.inf,
            "median": np.inf,
            "p90": np.inf,
            "fail_rate": 1.0,
            "n_ok": 0,
            "mu_true": np.array([], dtype=float),
            "mu_hat": np.array([], dtype=float),
        }

    mean = float(np.mean(errs))
    median = float(np.median(errs))
    p90 = float(np.quantile(errs, 0.90))
    fail_rate = float(fail / max(1, len(curves)))
    score = 0.50 * median + 0.35 * mean + 0.15 * p90 + 0.05 * fail_rate
    return {
        "score": score,
        "mean": mean,
        "median": median,
        "p90": p90,
        "fail_rate": fail_rate,
        "n_ok": int(len(errs)),
        "mu_true": np.asarray(mu_true_list, dtype=float),
        "mu_hat": np.asarray(mu_hat_list, dtype=float),
    }


Using shared evaluation set: n=500
  mu_true range: q05=0.071, med=0.329, q95=1.652
  noise_sigma range: q05=0.0005, med=0.0100, q95=0.6632
  n_pts/t_span/od_fold medians: n_pts=514.0, t_span=51.64, od_fold=54.08
  n_pts/t_span ranges: n_pts=[100, 996], t_span=[24.07, 95.62] h
  model counts: {'gompertz': 219, 'logistic': 166, 'richards': 115}


In [245]:
# ---------------------------------------------------------------------------
# Single-dataset evaluation: baseline (curve-local) vs multiple auto variants
# Outlier-robust comparison: fit methods on the same curves, then drop catastrophic tails.
# ---------------------------------------------------------------------------

if "eval_curves" not in globals() or len(eval_curves) == 0:
    raise RuntimeError("Run the evaluation-set preparation cell first.")


# Outlier filtering controls (relative-error units: 1.0 == 100%).
ERR_FILTER_MODE = str(globals().get("ERR_FILTER_MODE", "iqr")).strip().lower()
ERR_FILTER_IQR_K = float(globals().get("ERR_FILTER_IQR_K", 3.0))
ERR_FILTER_QUANTILE = float(globals().get("ERR_FILTER_QUANTILE", 0.995))
ERR_FILTER_MAX_REL_ERR = float(globals().get("ERR_FILTER_MAX_REL_ERR", np.inf))
ERR_FILTER_MIN_KEEP_FRAC = float(globals().get("ERR_FILTER_MIN_KEEP_FRAC", 0.70))
ERR_FILTER_MIN_KEEP_N = int(globals().get("ERR_FILTER_MIN_KEEP_N", 100))


AUTO_VARIANTS = [
    {
        "key": "auto_default",
        "label": "Auto\n(default)",
        "short": "Auto default",
        "color": "#1f77b4",
        "sigma_method": "second_diff_mad",
        "trim_mad": True,
        "clip_smoothing": True,
    },
    {
        "key": "auto_no_trim",
        "label": "Auto\n(no MAD trim)",
        "short": "No MAD trim",
        "color": "#17becf",
        "sigma_method": "second_diff_mad",
        "trim_mad": False,
        "clip_smoothing": True,
    },
    {
        "key": "auto_no_clip",
        "label": "Auto\n(no clipping)",
        "short": "No clipping",
        "color": "#ff7f0e",
        "sigma_method": "second_diff_mad",
        "trim_mad": True,
        "clip_smoothing": False,
    },
    {
        "key": "auto_no_trim_no_clip",
        "label": "Auto\n(no trim + no clip)",
        "short": "No trim/no clip",
        "color": "#9467bd",
        "sigma_method": "second_diff_mad",
        "trim_mad": False,
        "clip_smoothing": False,
    },
    {
        "key": "auto_first_diff_mad",
        "label": "Auto\n(first-diff MAD)",
        "short": "First-diff MAD",
        "color": "#2ca02c",
        "sigma_method": "first_diff_mad",
        "trim_mad": True,
        "clip_smoothing": True,
    },
    {
        "key": "auto_second_diff_iqr",
        "label": "Auto\n(second-diff IQR)",
        "short": "Second-diff IQR",
        "color": "#8c564b",
        "sigma_method": "second_diff_iqr",
        "trim_mad": False,
        "clip_smoothing": True,
    },
    {
        "key": "auto_second_diff_std",
        "label": "Auto\n(second-diff STD)",
        "short": "Second-diff STD",
        "color": "#e377c2",
        "sigma_method": "second_diff_std",
        "trim_mad": False,
        "clip_smoothing": True,
    },
]

AUTO_VARIANTS = list(globals().get("AUTO_VARIANTS", AUTO_VARIANTS))
variant_by_key = {str(v["key"]): v for v in AUTO_VARIANTS}
if "auto_default" not in variant_by_key:
    raise RuntimeError("AUTO_VARIANTS must include a variant with key='auto_default'.")


def _mean_ci95(vals):
    vals = np.asarray(vals, dtype=float)
    vals = vals[np.isfinite(vals)]
    n = len(vals)
    if n == 0:
        return np.nan, np.nan, np.nan, 0
    m = float(np.mean(vals))
    if n < 2:
        return m, np.nan, np.nan, n
    sd = float(np.std(vals, ddof=1))
    ci = 1.96 * sd / np.sqrt(n)
    return m, sd, ci, n


def _r_squared(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    if np.sum(mask) < 2:
        return np.nan
    yt = y_true[mask]
    yp = y_pred[mask]
    ss_tot = float(np.sum((yt - np.mean(yt)) ** 2))
    if ss_tot <= 0:
        return np.nan
    ss_res = float(np.sum((yt - yp) ** 2))
    return float(1.0 - ss_res / ss_tot)


# Curve-local baseline: each curve gets its own sigma_i and s_i = n_i * sigma_i^2.
def _baseline_local_noise_stats(t, N):
    t = np.asarray(t, dtype=float)
    N = np.asarray(N, dtype=float)
    if len(t) < 4 or np.ptp(t) <= 0:
        raise ValueError("Curve must have >=4 points and non-zero time span")

    y = np.log(np.clip(N, 1e-12, None))
    n = max(len(t), 2)

    sigma_i = _robust_sigma_from_second_diff(t, y, trim_q=0.80)
    if (not np.isfinite(sigma_i)) or (sigma_i <= 0):
        sigma_i = 1e-4

    s_min = 0.01
    s_max = max(s_min * 10, 0.8 * float(n) * float(np.var(y)))
    s_i = float(np.clip(float(n) * float(sigma_i) ** 2, s_min, s_max))
    return s_i, float(sigma_i)


def _build_joint_keep_mask(err_base, err_auto):
    err_base = np.asarray(err_base, dtype=float)
    err_auto = np.asarray(err_auto, dtype=float)

    valid = np.isfinite(err_base) & np.isfinite(err_auto) & (err_base >= 0) & (err_auto >= 0)
    if np.sum(valid) == 0:
        return valid, np.inf, "none (no valid paired errors)"

    keep = valid.copy()
    rule_bits = []

    if np.isfinite(ERR_FILTER_MAX_REL_ERR):
        keep &= (err_base <= ERR_FILTER_MAX_REL_ERR) & (err_auto <= ERR_FILTER_MAX_REL_ERR)
        rule_bits.append(f"abs cap <= {ERR_FILTER_MAX_REL_ERR:.4g}")

    upper_rel = np.inf
    if ERR_FILTER_MODE == "none":
        rule_bits.append("mode=none")
    elif ERR_FILTER_MODE == "iqr":
        pooled = np.log1p(np.concatenate([err_base[valid], err_auto[valid]]))
        q1, q3 = np.quantile(pooled, [0.25, 0.75])
        iqr = max(float(q3 - q1), 1e-12)
        upper_log = float(q3 + ERR_FILTER_IQR_K * iqr)
        upper_rel = float(np.expm1(upper_log))
        keep &= (err_base <= upper_rel) & (err_auto <= upper_rel)
        rule_bits.append(f"mode=iqr(log1p), <= {upper_rel:.4g}")
    elif ERR_FILTER_MODE == "quantile":
        qq = float(np.clip(ERR_FILTER_QUANTILE, 0.80, 0.9999))
        upper_rel = float(np.quantile(np.concatenate([err_base[valid], err_auto[valid]]), qq))
        keep &= (err_base <= upper_rel) & (err_auto <= upper_rel)
        rule_bits.append(f"mode=quantile({qq:.3f}), <= {upper_rel:.4g}")
    else:
        raise ValueError("ERR_FILTER_MODE must be one of: 'iqr', 'quantile', 'none'.")

    valid_n = int(np.sum(valid))
    min_keep = max(int(np.ceil(ERR_FILTER_MIN_KEEP_FRAC * valid_n)), int(ERR_FILTER_MIN_KEEP_N))
    min_keep = min(min_keep, valid_n)

    if int(np.sum(keep)) < min_keep:
        keep = valid.copy()
        rule_bits.append(f"fallback=disabled_filter(min_keep={min_keep})")

    return keep, upper_rel, ", ".join(rule_bits)


# Paired evaluation: all methods on the same curve before any filtering.
paired_records = []
for rec in eval_curves:
    t = rec["t"]
    N_obs = rec["N_obs"]
    mu_true = float(rec["mu_true"])

    try:
        s_i, sigma_i = _baseline_local_noise_stats(t, N_obs)
        sp_base = fixed_s_spline(t, N_obs, s=s_i)
        mu_base = _mu_max_from_spline(sp_base, t, edge_frac=0.0, q=1.0, n_eval=420)
    except Exception:
        continue

    if (not np.isfinite(mu_true)) or (mu_true <= 0):
        continue
    if (not np.isfinite(mu_base)) or (mu_base <= 0):
        continue

    auto = {}
    for v in AUTO_VARIANTS:
        try:
            sp_v, s_v, sigma_v = auto_smooth_spline_variant(
                t,
                N_obs,
                smooth_mult=float(_SMOOTH_MULT),
                sigma_method=str(v.get("sigma_method", "second_diff_mad")),
                trim_mad=bool(v.get("trim_mad", True)),
                clip_smoothing=bool(v.get("clip_smoothing", True)),
            )
            mu_v = _mu_max_from_spline(sp_v, t, edge_frac=0.0, q=1.0, n_eval=420)
        except Exception:
            continue

        if (not np.isfinite(mu_v)) or (mu_v <= 0):
            continue

        auto[str(v["key"])] = {
            "mu_hat": float(mu_v),
            "s": float(s_v),
            "sigma": float(sigma_v),
        }

    if "auto_default" not in auto:
        continue

    paired_records.append({
        "mu_true": mu_true,
        "mu_base": float(mu_base),
        "s_base": float(s_i),
        "sigma_base": float(sigma_i),
        "auto": auto,
        "is_stress": bool(rec.get("is_stress", False)),
        "stress_tag": str(rec.get("stress_tag", "")),
        "target_fail": str(rec.get("target_fail", "none")),
    })

if not paired_records:
    raise RuntimeError("No valid paired method fits were produced on eval_curves.")

mu_true_all = np.asarray([r["mu_true"] for r in paired_records], dtype=float)
mu_base_all = np.asarray([r["mu_base"] for r in paired_records], dtype=float)
errs_fixed_full = np.asarray([abs(r["mu_base"] - r["mu_true"]) / (r["mu_true"] + 1e-8) for r in paired_records], dtype=float)
baseline_s_values_all = np.asarray([r["s_base"] for r in paired_records], dtype=float)
baseline_sigma_values_all = np.asarray([r["sigma_base"] for r in paired_records], dtype=float)
is_stress_all = np.asarray([bool(r["is_stress"]) for r in paired_records], dtype=bool)
stress_tag_all = np.asarray([str(r["stress_tag"]) for r in paired_records], dtype=object)
target_fail_all = np.asarray([str(r["target_fail"]) for r in paired_records], dtype=object)

auto_mu_all = {}
auto_errs_full = {}
auto_s_all = {}
auto_sigma_all = {}
for v in AUTO_VARIANTS:
    k = str(v["key"])
    mu_hat = np.asarray([r["auto"][k]["mu_hat"] if k in r["auto"] else np.nan for r in paired_records], dtype=float)
    s_val = np.asarray([r["auto"][k]["s"] if k in r["auto"] else np.nan for r in paired_records], dtype=float)
    sig_val = np.asarray([r["auto"][k]["sigma"] if k in r["auto"] else np.nan for r in paired_records], dtype=float)

    err_val = np.full_like(mu_true_all, np.nan, dtype=float)
    ok = np.isfinite(mu_hat) & (mu_hat > 0)
    err_val[ok] = np.abs(mu_hat[ok] - mu_true_all[ok]) / (mu_true_all[ok] + 1e-8)

    auto_mu_all[k] = mu_hat
    auto_errs_full[k] = err_val
    auto_s_all[k] = s_val
    auto_sigma_all[k] = sig_val

errs_default_full = auto_errs_full["auto_default"]
keep_mask, err_cutoff, filter_rule = _build_joint_keep_mask(errs_fixed_full, errs_default_full)
if int(np.sum(keep_mask)) < 2:
    raise RuntimeError(
        "Too few curves remain after filtering. Relax ERR_FILTER_* settings or set ERR_FILTER_MODE='none'."
    )

mu_true = mu_true_all[keep_mask]
mu_base = mu_base_all[keep_mask]
errs_fixed = errs_fixed_full[keep_mask]
baseline_s_values = baseline_s_values_all[keep_mask]
baseline_sigma_values = baseline_sigma_values_all[keep_mask]
is_stress = is_stress_all[keep_mask]
stress_tag = stress_tag_all[keep_mask]
target_fail = target_fail_all[keep_mask]

methods = []
m_f, sd_f, ci_f, n_f = _mean_ci95(errs_fixed)
methods.append({
    "key": "baseline",
    "label": "Baseline\n(curve-local)",
    "short": "Baseline",
    "color": "red",
    "res": {"mu_true": mu_true, "mu_hat": mu_base},
    "errs": errs_fixed,
    "mean": m_f,
    "ci": ci_f,
    "n": n_f,
    "r2": _r_squared(mu_true, mu_base),
    "mask": keep_mask.copy(),
    "pair_x": np.array([], dtype=float),
    "pair_y": np.array([], dtype=float),
    "s_values": baseline_s_values,
    "sigma_values": baseline_sigma_values,
})

for v in AUTO_VARIANTS:
    k = str(v["key"])
    mu_hat_full = auto_mu_all[k]
    err_full = auto_errs_full[k]
    s_full = auto_s_all[k]
    sigma_full = auto_sigma_all[k]

    mask_v = keep_mask & np.isfinite(mu_hat_full) & (mu_hat_full > 0) & np.isfinite(err_full) & (err_full >= 0)

    mu_true_v = mu_true_all[mask_v]
    mu_hat_v = mu_hat_full[mask_v]
    errs_v = err_full[mask_v]
    s_v = s_full[mask_v]
    sigma_v = sigma_full[mask_v]

    mv, sdv, civ, nv = _mean_ci95(errs_v)
    methods.append({
        "key": k,
        "label": str(v.get("label", k)),
        "short": str(v.get("short", k)),
        "color": str(v.get("color", "blue")),
        "res": {"mu_true": mu_true_v, "mu_hat": mu_hat_v},
        "errs": errs_v,
        "mean": mv,
        "ci": civ,
        "n": nv,
        "r2": _r_squared(mu_true_v, mu_hat_v),
        "mask": mask_v,
        "pair_x": errs_fixed_full[mask_v] * 100.0,
        "pair_y": errs_v * 100.0,
        "s_values": s_v,
        "sigma_values": sigma_v,
    })


# One figure, four panels
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=[
        "Parity (filtered)",
        "mu_max relative error (filtered)",
        "Paired errors vs baseline (filtered)",
        "Distribution of smoothing factor s (filtered)",
    ],
    horizontal_spacing=0.12,
    vertical_spacing=0.16,
)

# 1) Parity
for m in methods:
    res = m["res"]
    if len(res["mu_true"]) > 0:
        fig.add_trace(go.Scatter(
            x=res["mu_true"],
            y=res["mu_hat"],
            mode="markers",
            marker=dict(size=6, color=m["color"], opacity=0.50),
            name=m["label"].replace("\n", " "),
        ), row=1, col=1)

all_true = np.concatenate([m["res"]["mu_true"] for m in methods if len(m["res"]["mu_true"]) > 0])
all_hat = np.concatenate([m["res"]["mu_hat"] for m in methods if len(m["res"]["mu_hat"]) > 0])
lo = float(min(np.min(all_true), np.min(all_hat)))
hi = float(max(np.max(all_true), np.max(all_hat)))
fig.add_trace(go.Scatter(
    x=[lo, hi],
    y=[lo, hi],
    mode="lines",
    line=dict(color="black", dash="dash"),
    name="Parity line",
    showlegend=False,
), row=1, col=1)

for i, m in enumerate(methods):
    y_pos = 0.98 - 0.06 * i
    r2_txt = f"{m['short']}: R^2={m['r2']:.4f}" if np.isfinite(m["r2"]) else f"{m['short']}: R^2=NA"
    fig.add_annotation(
        x=0.02,
        y=y_pos,
        xref="x domain",
        yref="y domain",
        text=r2_txt,
        showarrow=False,
        xanchor="left",
        yanchor="top",
        align="left",
        font=dict(color=m["color"], size=10),
    )

# 2) mu_max relative error
bar_labels = [f"{m['label'].replace(chr(10), ' ')} (n={m['n']})" for m in methods]
bar_means = [m["mean"] * 100.0 for m in methods]
bar_errors = [m["ci"] * 100.0 if np.isfinite(m["ci"]) else 0.0 for m in methods]
bar_colors = [m["color"] for m in methods]
fig.add_trace(go.Bar(
    x=bar_labels,
    y=bar_means,
    error_y=dict(type="data", array=bar_errors, visible=True),
    marker=dict(color=bar_colors),
    name="Mean +/- 95% CI",
    showlegend=False,
), row=1, col=2)

# 3) Paired errors vs baseline for each auto variant
pair_lim = 1.0
has_pairs = False
for m in methods[1:]:
    if len(m["pair_x"]) == 0:
        continue
    has_pairs = True
    pair_lim = max(pair_lim, float(np.max(m["pair_x"])), float(np.max(m["pair_y"])))
    fig.add_trace(go.Scatter(
        x=m["pair_x"],
        y=m["pair_y"],
        mode="markers",
        marker=dict(size=6, color=m["color"], opacity=0.45),
        name=f"{m['short']} vs baseline",
        showlegend=False,
    ), row=2, col=1)

if not has_pairs:
    print("No valid paired errors available for paired subplot.")

pair_lim = max(pair_lim, 1e-6)
fig.add_trace(go.Scatter(
    x=[0.0, pair_lim],
    y=[0.0, pair_lim],
    mode="lines",
    line=dict(color="black", dash="dash"),
    name="Paired parity",
    showlegend=False,
), row=2, col=1)

# 4) s distribution swarm
rng_swarm = np.random.default_rng(20260224)
s_groups = [("Baseline", baseline_s_values, "red")]
for m in methods[1:]:
    if len(m["s_values"]) > 0:
        s_groups.append((m["short"], m["s_values"], m["color"]))

for idx, (label, arr, color) in enumerate(s_groups):
    xj = rng_swarm.normal(loc=float(idx), scale=0.06, size=len(arr))
    fig.add_trace(go.Scatter(
        x=xj,
        y=arr,
        mode="markers",
        marker=dict(color=color, size=6, opacity=0.60),
        name=f"{label} s",
        showlegend=False,
    ), row=2, col=2)
    fig.add_trace(go.Scatter(
        x=[float(idx)],
        y=[float(np.mean(arr))],
        mode="markers",
        marker=dict(color=color, size=11, symbol="x"),
        name=f"{label} mean s",
        showlegend=False,
    ), row=2, col=2)

kept_n = int(np.sum(keep_mask))
all_n = int(len(keep_mask))
dropped_n = int(all_n - kept_n)

fig.update_layout(
    title=f"Baseline vs auto variants comparison (filtered paired dataset, n_variants={len(AUTO_VARIANTS)})",
    width=1400,
    height=980,
    template=PLOTLY_TEMPLATE,
)

fig.add_annotation(
    x=0.5,
    y=1.08,
    xref="paper",
    yref="paper",
    showarrow=False,
    text=f"Kept {kept_n}/{all_n} paired curves (dropped {dropped_n}); {filter_rule}",
    align="center",
    font=dict(size=11),
)

fig.update_xaxes(title_text="True mu_max", row=1, col=1)
fig.update_yaxes(title_text="Predicted mu_max", row=1, col=1)

fig.update_xaxes(title_text="Method", row=1, col=2)
fig.update_yaxes(title_text="mu_max relative error (%)", row=1, col=2)

fig.update_xaxes(title_text="Baseline relative error (%)", row=2, col=1)
fig.update_yaxes(title_text="Variant relative error (%)", row=2, col=1)

fig.update_xaxes(
    title_text="Method",
    row=2,
    col=2,
    tickmode="array",
    tickvals=[float(i) for i in range(len(s_groups))],
    ticktext=[g[0] for g in s_groups],
    range=[-0.5, float(len(s_groups) - 0.5)],
)
fig.update_yaxes(title_text="s value", row=2, col=2)

fig.show(config=PLOTLY_CONFIG)


print("Outlier-filter configuration")
print(f"  mode={ERR_FILTER_MODE}, iqr_k={ERR_FILTER_IQR_K}, quantile={ERR_FILTER_QUANTILE:.4f}, max_rel_err={ERR_FILTER_MAX_REL_ERR}")
print(f"  kept={kept_n}/{all_n}, dropped={dropped_n}")
if np.isfinite(err_cutoff):
    print(f"  derived cutoff (relative error): {err_cutoff:.6g} ({100.0 * err_cutoff:.2f}%)")
print(f"  applied rule: {filter_rule}")

print("Method comparison summary (filtered paired set)")
print(f"  {'Method':<32} {'mean':>8} {'median':>8} {'p90':>8} {'R^2':>8} {'n':>6}")
print(f"  {'-'*80}")
for m in methods:
    errs = m["errs"]
    med = float(np.median(errs)) if len(errs) else np.nan
    p90 = float(np.quantile(errs, 0.90)) if len(errs) else np.nan
    r2 = m["r2"]
    lbl = m["label"].replace("\n", " ")
    r2_txt = f"{r2:8.4f}" if np.isfinite(r2) else "      nan"
    print(f"  {lbl:<32} {m['mean']:>8.4f} {med:>8.4f} {p90:>8.4f} {r2_txt} {m['n']:>6d}")

print("Unfiltered reference (same paired curves, before outlier removal)")
print(
    f"  baseline mean={np.mean(errs_fixed_full):.4f}, median={np.median(errs_fixed_full):.4f}, "
    f"p90={np.quantile(errs_fixed_full,0.90):.4f}, R^2={_r_squared(mu_true_all, mu_base_all):.4f}"
)
for v in AUTO_VARIANTS:
    k = str(v["key"])
    err_full = auto_errs_full[k]
    mu_hat_full = auto_mu_all[k]
    valid = np.isfinite(err_full) & np.isfinite(mu_hat_full) & (mu_hat_full > 0)
    if np.sum(valid) == 0:
        print(f"  {k:<16} no valid fits")
        continue
    print(
        f"  {k:<16} mean={np.mean(err_full[valid]):.4f}, median={np.median(err_full[valid]):.4f}, "
        f"p90={np.quantile(err_full[valid],0.90):.4f}, R^2={_r_squared(mu_true_all[valid], mu_hat_full[valid]):.4f}"
    )

if len(baseline_sigma_values) > 0:
    print(
        "Curve-local baseline sigma summary (filtered): "
        f"median={np.median(baseline_sigma_values):.6g}, "
        f"q25={np.quantile(baseline_sigma_values, 0.25):.6g}, "
        f"q75={np.quantile(baseline_sigma_values, 0.75):.6g}"
    )

print("s means (filtered):")
print(f"  baseline={np.mean(baseline_s_values):.6g}")
for m in methods[1:]:
    if len(m["s_values"]) == 0:
        print(f"  {m['key']:<16}=nan")
    else:
        print(f"  {m['key']:<16}={np.mean(m['s_values']):.6g}")



Outlier-filter configuration
  mode=iqr, iqr_k=3.0, quantile=0.9950, max_rel_err=inf
  kept=247/317, dropped=70
  derived cutoff (relative error): 3.75153 (375.15%)
  applied rule: mode=iqr(log1p), <= 3.752
Method comparison summary (filtered paired set)
  Method                           mean   median      p90      R^2      n
  ------------------------------------------------------------------------
  Baseline (curve-local)         0.2286   0.0383   0.6508   0.5841    247
  Auto (default)                 0.1689   0.0376   0.3477   0.7287    247
Unfiltered reference (same paired curves, before outlier removal)
  baseline mean=2101.4507, median=0.0651, p90=48.0319, R^2=-558156351.8147
  auto     mean=2085.0722, median=0.0600, p90=5.4494, R^2=-558150433.5972
Targeted stress-case summary (unfiltered paired set)
  target_fail=baseline n=  37 | baseline_mean=17923.6157, auto_mean=17785.6278, delta(auto-baseline)=-137.9880
  target_fail=auto     n=  11 | baseline_mean=27.4322, auto_mean=27.2

In [ ]:
# ---------------------------------------------------------------------------
# Runtime benchmark: baseline vs auto variants
# ---------------------------------------------------------------------------

if "eval_curves" not in globals() or len(eval_curves) == 0:
    raise RuntimeError("Run the evaluation-set preparation cell first.")

import time

if "AUTO_VARIANTS" not in globals() or len(AUTO_VARIANTS) == 0:
    AUTO_VARIANTS = [
        {
            "key": "auto_default",
            "label": "Auto\n(default)",
            "short": "Auto default",
            "color": "#1f77b4",
            "sigma_method": "second_diff_mad",
            "trim_mad": True,
            "clip_smoothing": True,
        }
    ]

RUNTIME_BENCH_N = int(globals().get("RUNTIME_BENCH_N", min(len(eval_curves), 200)))
RUNTIME_BENCH_REPEATS = int(globals().get("RUNTIME_BENCH_REPEATS", 3))
RUNTIME_BENCH_WARMUP = bool(globals().get("RUNTIME_BENCH_WARMUP", True))
RUNTIME_BENCH_SEED = int(globals().get("RUNTIME_BENCH_SEED", 20260224))

RUNTIME_BENCH_N = int(np.clip(RUNTIME_BENCH_N, 1, len(eval_curves)))
RUNTIME_BENCH_REPEATS = int(max(1, RUNTIME_BENCH_REPEATS))

rng_rt = np.random.default_rng(RUNTIME_BENCH_SEED)
bench_idx = np.sort(rng_rt.choice(len(eval_curves), size=RUNTIME_BENCH_N, replace=False))

method_specs = [
    {
        "key": "baseline",
        "label": "Baseline",
        "short": "Baseline",
        "color": "red",
        "kind": "baseline",
    }
]

for v in AUTO_VARIANTS:
    method_specs.append({
        "key": str(v["key"]),
        "label": str(v.get("label", v["key"])).replace("\n", " "),
        "short": str(v.get("short", v["key"])),
        "color": str(v.get("color", "blue")),
        "kind": "auto",
        "sigma_method": str(v.get("sigma_method", "second_diff_mad")),
        "trim_mad": bool(v.get("trim_mad", True)),
        "clip_smoothing": bool(v.get("clip_smoothing", True)),
    })


def _fit_mu_for_runtime(spec, t, N_obs):
    if spec["kind"] == "baseline":
        s_i, _ = _baseline_local_noise_stats(t, N_obs)
        sp = fixed_s_spline(t, N_obs, s=s_i)
    else:
        sp, _, _ = auto_smooth_spline_variant(
            t,
            N_obs,
            smooth_mult=float(_SMOOTH_MULT),
            sigma_method=spec["sigma_method"],
            trim_mad=spec["trim_mad"],
            clip_smoothing=spec["clip_smoothing"],
        )

    mu = _mu_max_from_spline(sp, t, edge_frac=0.0, q=1.0, n_eval=420)
    return float(mu)


if RUNTIME_BENCH_WARMUP and len(bench_idx) > 0:
    rec0 = eval_curves[int(bench_idx[0])]
    for spec in method_specs:
        try:
            _ = _fit_mu_for_runtime(spec, rec0["t"], rec0["N_obs"])
        except Exception:
            pass

runtime_rows = []

for spec in method_specs:
    samples_s = []
    ok = 0
    fail = 0

    t_method_start = time.perf_counter()
    for _ in range(RUNTIME_BENCH_REPEATS):
        for idx in bench_idx:
            rec = eval_curves[int(idx)]
            t = rec["t"]
            N_obs = rec["N_obs"]

            t0 = time.perf_counter()
            valid = False
            try:
                mu_hat = _fit_mu_for_runtime(spec, t, N_obs)
                valid = bool(np.isfinite(mu_hat) and (mu_hat > 0))
            except Exception:
                valid = False
            dt = time.perf_counter() - t0

            if np.isfinite(dt) and dt >= 0:
                samples_s.append(float(dt))

            if valid:
                ok += 1
            else:
                fail += 1

    total_wall_s = time.perf_counter() - t_method_start
    arr_s = np.asarray(samples_s, dtype=float)

    if len(arr_s) == 0:
        mean_ms = np.inf
        median_ms = np.inf
        p90_ms = np.inf
        ci95_ms = np.nan
    else:
        mean_ms = float(np.mean(arr_s) * 1000.0)
        median_ms = float(np.median(arr_s) * 1000.0)
        p90_ms = float(np.quantile(arr_s, 0.90) * 1000.0)
        if len(arr_s) > 1:
            sem_s = float(np.std(arr_s, ddof=1) / np.sqrt(len(arr_s)))
            ci95_ms = float(1.96 * sem_s * 1000.0)
        else:
            ci95_ms = np.nan

    n_calls = int(len(arr_s))
    ok_rate = float(100.0 * ok / max(1, ok + fail))

    runtime_rows.append({
        "key": spec["key"],
        "label": spec["label"],
        "short": spec["short"],
        "color": spec["color"],
        "n_calls": n_calls,
        "ok": int(ok),
        "fail": int(fail),
        "ok_rate": ok_rate,
        "mean_ms": mean_ms,
        "median_ms": median_ms,
        "p90_ms": p90_ms,
        "ci95_ms": ci95_ms,
        "total_wall_s": float(total_wall_s),
        "samples_ms": arr_s * 1000.0,
    })

# Sort for plotting/reporting by mean runtime.
runtime_rows = sorted(runtime_rows, key=lambda r: (np.isfinite(r["mean_ms"]) == 0, r["mean_ms"]))
runtime_benchmark_results = runtime_rows

print("Runtime benchmark configuration")
print(
    f"  n_curves={RUNTIME_BENCH_N}, repeats={RUNTIME_BENCH_REPEATS}, "
    f"total method calls each={RUNTIME_BENCH_N * RUNTIME_BENCH_REPEATS}"
)
print("Runtime summary (sorted by mean ms per fit)")
print(f"  {'Method':<28} {'mean_ms':>10} {'median_ms':>10} {'p90_ms':>10} {'ok%':>8} {'calls':>8}")
print(f"  {'-'*86}")
for r in runtime_rows:
    print(
        f"  {r['label']:<28} {r['mean_ms']:>10.4f} {r['median_ms']:>10.4f} "
        f"{r['p90_ms']:>10.4f} {r['ok_rate']:>8.2f} {r['n_calls']:>8d}"
    )

bar_x = [r["label"] for r in runtime_rows]
bar_y = [r["mean_ms"] for r in runtime_rows]
bar_err = [0.0 if not np.isfinite(r["ci95_ms"]) else r["ci95_ms"] for r in runtime_rows]
bar_color = [r["color"] for r in runtime_rows]

fig_rt = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=[
        "Mean runtime per fit (ms)",
        "Runtime distribution per fit (ms)",
    ],
    horizontal_spacing=0.14,
)

fig_rt.add_trace(
    go.Bar(
        x=bar_x,
        y=bar_y,
        error_y=dict(type="data", array=bar_err, visible=True),
        marker=dict(color=bar_color),
        name="Mean +/- 95% CI",
        showlegend=False,
    ),
    row=1,
    col=1,
)

for r in runtime_rows:
    samp = np.asarray(r["samples_ms"], dtype=float)
    if len(samp) == 0:
        continue
    fig_rt.add_trace(
        go.Box(
            y=samp,
            name=r["label"],
            marker_color=r["color"],
            boxmean=True,
            jitter=0.25,
            pointpos=0.0,
            showlegend=False,
        ),
        row=1,
        col=2,
    )

# Optional log-scale if runtime spread is very large.
finite_means = np.asarray([v for v in bar_y if np.isfinite(v) and v > 0], dtype=float)
use_log = len(finite_means) > 1 and (float(np.max(finite_means)) / float(np.min(finite_means)) > 20.0)

fig_rt.update_yaxes(title_text="Runtime (ms)", row=1, col=1, type="log" if use_log else "linear")
fig_rt.update_yaxes(title_text="Runtime (ms)", row=1, col=2, type="log" if use_log else "linear")

fig_rt.update_xaxes(title_text="Method", row=1, col=1)
fig_rt.update_xaxes(title_text="Method", row=1, col=2)

fig_rt.update_layout(
    title=(
        "Runtime benchmark across methods "
        f"(n_curves={RUNTIME_BENCH_N}, repeats={RUNTIME_BENCH_REPEATS}, "
        f"calls/method={RUNTIME_BENCH_N * RUNTIME_BENCH_REPEATS})"
    ),
    width=1450,
    height=540,
    template=PLOTLY_TEMPLATE,
)

fig_rt.show(config=PLOTLY_CONFIG)

